Esta notebook parte tomando el dataframe generado en la notebook anterior. Observamos que un porcentaje pequeño de tarjetas tiene más de un contrato y se decide eliminarlas del dataset final. A su vez, se transforma la columna de Time al formato datime para que se puedan realizar operaciones sobre dicha columna. Finalmente, se genera un dataset que cuenta con las siguientes caracterísitcas:
 - Cada fila representa un número de tarjeta único y con un único contrato.
 - En las columnas generamos: cantidad de viajes, cantidad de líneas distintas de colectivo que utiliza el pasajero, línea de mayor frecuencia, cantidad de días distintos de viaje, primer y último día de viaje, primera hora, última hora y hora promedio de viaje.
 
Nuevamente, se exporta este dataset a csv que es el que se utilizará como input en el notebook siguiente

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
from datetime import datetime



In [2]:
# partimos del csv del documento anterior
df = pd.read_csv("../Mentoria_diplomatura/base_mayo22_habil_v2.csv")

In [3]:
df.head()

,LINEA,SENTIDO,FECHA,TARJETA,CONTRATO,TIME,DIA
0,L42 E,Ida,02/05/2022 04:36:14 PM,1000001,Usuario comun,16:36:14,02/05/2022
1,L35 T,Ida,02/05/2022 10:22:27 PM,1000002,Usuario comun,22:22:27,02/05/2022
2,C,Ida,02/05/2022 03:20:12 PM,1000003,Usuario comun,15:20:12,02/05/2022
3,C,Vuelta,02/05/2022 09:18:06 AM,1000003,Usuario comun,09:18:06,02/05/2022
4,C,Ida,02/05/2022 06:56:40 AM,1000004,Usuario comun,06:56:40,02/05/2022


In [4]:
df.shape

(8430650, 7)

In [5]:
df.TARJETA.nunique()

698084

In [6]:
df.LINEA.nunique()

71

In [50]:
df.DIA.value_counts()

04/05/2022    455802
12/05/2022    454585
06/05/2022    454366
03/05/2022    453346
05/05/2022    451424
10/05/2022    448905
13/05/2022    448231
23/05/2022    447513
24/05/2022    446953
09/05/2022    445510
20/05/2022    442949
16/05/2022    437444
02/05/2022    434103
27/05/2022    433996
31/05/2022    431913
19/05/2022    425781
11/05/2022    425759
26/05/2022    424968
30/05/2022    424639
18/05/2022     37000
07/05/2022      1066
21/05/2022      1052
14/05/2022       969
28/05/2022       874
25/05/2022       730
01/06/2022       697
17/05/2022        43
29/05/2022        32
Name: DIA, dtype: int64

In [7]:
df.CONTRATO.nunique()

7

In [9]:
df.DIA.nunique()

28

In [10]:
df.dtypes

LINEA       object
SENTIDO     object
FECHA       object
TARJETA      int64
CONTRATO    object
TIME        object
DIA         object
dtype: object

In [27]:
df2 = df.groupby('TARJETA')['CONTRATO'].nunique().reset_index()

In [28]:
# 9192 tarjetas tienen 2 contratos
df2.CONTRATO.value_counts()

1    688892
2      9192
Name: CONTRATO, dtype: int64

In [29]:
df2[df2['CONTRATO']>1]['CONTRATO'].count() /  df2['CONTRATO'].count()

0.013167469817385874

Solo el 1.3% de las tarjetas tienen más de un contrato, por lo que no se tomarán en cuenta para el anáisis.

In [30]:
# definimos una nueva columna que contiene 1 si la tarjeta tiene más de un contrato y 0 si la tarjeta tiene solo un contrato
df2['mascontratos'] = np.where(df2['CONTRATO']>1, 1, 0)

In [31]:
df2[df2['mascontratos']==1]

,TARJETA,CONTRATO,mascontratos
2983,1002984,2,1
3071,1003072,2,1
5483,1005484,2,1
5517,1005518,2,1
46014,1046015,2,1
...,...,...,...
687518,1687519,2,1
687527,1687528,2,1
687814,1687815,2,1
695117,1695118,2,1


In [32]:
tarjetas_2 = df2[df2['mascontratos']==1]['TARJETA'].to_list()

In [33]:
# lista de tarjetas que tienen más de un contrato
tarjetas_2

[1002984,
 1003072,
 1005484,
 1005518,
 1046015,
 1046051,
 1046077,
 1046213,
 1046217,
 1046297,
 1046323,
 1046327,
 1046367,
 1046373,
 1046385,
 1046397,
 1046404,
 1046519,
 1046663,
 1046665,
 1046707,
 1046733,
 1046747,
 1046822,
 1046928,
 1047012,
 1047116,
 1047180,
 1047244,
 1047279,
 1047312,
 1047347,
 1047364,
 1047370,
 1073297,
 1073429,
 1073527,
 1073627,
 1073637,
 1073662,
 1073762,
 1073764,
 1073952,
 1073972,
 1074092,
 1074109,
 1074132,
 1074141,
 1074214,
 1090846,
 1090895,
 1091008,
 1091126,
 1091168,
 1091220,
 1091359,
 1097856,
 1097859,
 1097867,
 1097873,
 1098008,
 1098143,
 1098205,
 1098263,
 1098346,
 1098352,
 1098361,
 1098383,
 1098465,
 1098487,
 1098523,
 1098539,
 1098608,
 1110560,
 1110593,
 1110638,
 1110781,
 1110854,
 1110863,
 1110887,
 1110899,
 1110906,
 1111014,
 1111103,
 1111118,
 1111190,
 1111243,
 1111263,
 1111291,
 1111305,
 1111368,
 1111382,
 1111475,
 1111510,
 1111584,
 1111608,
 1111616,
 1111628,
 1111641,
 1111649,


In [34]:
# Creamos un nuevo dataset que no incluye las tarjetas con más de un contrato
df_clean = df[~df['TARJETA'].isin(tarjetas_2)]

In [35]:
df_clean.shape

(8166309, 7)

In [36]:
df_clean.head()

,LINEA,SENTIDO,FECHA,TARJETA,CONTRATO,TIME,DIA
0,L42 E,Ida,02/05/2022 04:36:14 PM,1000001,Usuario comun,16:36:14,02/05/2022
1,L35 T,Ida,02/05/2022 10:22:27 PM,1000002,Usuario comun,22:22:27,02/05/2022
2,C,Ida,02/05/2022 03:20:12 PM,1000003,Usuario comun,15:20:12,02/05/2022
3,C,Vuelta,02/05/2022 09:18:06 AM,1000003,Usuario comun,09:18:06,02/05/2022
4,C,Ida,02/05/2022 06:56:40 AM,1000004,Usuario comun,06:56:40,02/05/2022


In [37]:
# Le damos el formato datetime a la columna TIME para luego poder hacer cálculos sobre esa columna
df_clean['TIME'] =  pd.to_datetime(df_clean['TIME'])

/tmp/ipykernel_153/3305669969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['TIME'] =  pd.to_datetime(df_clean['TIME'])


In [39]:
#Creamos el nuevo dataset que tiene en cada fila un número de tarjeta distinto y, en cada columna,
#algunas variables que van a servir para intentar predecir el contrato 
df_clean = df_clean.groupby(['TARJETA', 'CONTRATO']).agg({'TARJETA': 'count', 
                                               'LINEA':['nunique', lambda x:stats.mode(x)[0]],
                                               'DIA':['nunique', 'first', 'last'],
                                             'TIME':['first', 'last','mean']})

In [45]:
df_clean = df_clean.reset_index()

In [46]:
df_clean.head()

TARJETA       CONTRATO TARJETA   LINEA                DIA              \
                            count nunique <lambda_0> nunique       first   
0  1000001  Usuario comun      20       7      [L25]       7  02/05/2022   
1  1000002  Usuario comun       4       2    [L35 T]       3  02/05/2022   
2  1000003  Usuario comun      22       4        [C]      10  02/05/2022   
3  1000004  Usuario comun      52       3        [C]      19  02/05/2022   
4  1000005  Usuario comun      20       4      [L74]      18  02/05/2022   

                             TIME                      \
         last               first                last   
0  30/05/2022 2022-08-15 16:36:14 2022-08-15 21:31:39   
1  05/05/2022 2022-08-15 22:22:27 2022-08-15 20:11:31   
2  30/05/2022 2022-08-15 15:20:12 2022-08-15 09:16:09   
3  31/05/2022 2022-08-15 06:56:40 2022-08-15 22:08:36   
4  31/05/2022 2022-08-15 06:03:45 2022-08-15 05:58:58   

                                 
                           mean  
0 2022-08-15 19:34:14.900000000  
1 2022-08-15 20:49:39.000000000  
2 2022-08-15 11:31:15.954545408  
3 2022-08-15 14:51:13.461538560  
4 2022-08-15 06:49:33.849999872

In [47]:
df_clean.shape

(688892, 11)

In [51]:
# Exportamos el nuevo dataset a un archivo csv.
df_clean.to_csv("../Mentoria_diplomatura/base_mayo22_habil_v3.csv", index=False)